# LoreSmith

**LoreSmith** is an AI-powered search and retrieval system designed to help Dungeon Masters and players quickly uncover the rules, spells, and monsters hidden within the Dungeons & Dragons 5e System Reference Document (SRD). By combining semantic embeddings, hybrid search, and generative summarization, LoreSmith transforms raw SRD text into fast, accurate, and citation-grounded answers.

**Forge Fast Answers**: Ask natural questions like “Which CR 5 monsters resist poison?” or “What level 3 spells can heal multiple allies?” and get concise, structured answers.

**Rule-Accurate Guidance**: Every response is grounded in official SRD text, with citations for transparency.

**Choose-Your-Own-Adventure Search**: After each answer, LoreSmith suggests next steps—filter by class, level, challenge rating, environment, or even compare options side by side.

Built for Table Use: LoreSmith saves time during play sessions by replacing page-flipping with instant, context-rich search.

With LoreSmith, Dungeon Masters can focus less on rules lookups and more on storytelling, while players gain a reliable oracle to guide their adventures.

In [1]:
# CLEANROOM INSTALL (run in a fresh kernel; OK to rerun)
!pip uninstall -y keras tensorflow tensorflow-intel tf-keras protobuf || true

# CPU-only PyTorch + pinned, TF-free stack + protobuf<5
!pip install --upgrade --no-cache-dir \
  torch torchvision torchaudio \
  "protobuf<5" \
  "transformers==4.43.3" \
  "sentence-transformers==2.7.0" \
  "chromadb==0.5.5" \
  "rank_bm25==0.2.2" \
  "ipywidgets==8.1.2" \
  "tabulate==0.9.0" \
  "openai>=1.40.0"

# (Classic Notebook users only) enable widgets
# In JupyterLab 3+ you don't need this.
!jupyter nbextension enable --py widgetsnbextension -y || true

INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce 

### Setup

In [2]:
import os, json, math, hashlib, textwrap
from dataclasses import dataclass
from typing import List, Dict, Any, Optional

import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from IPython.display import display, Markdown, HTML
import ipywidgets as W
from tabulate import tabulate

import pandas as pd

In [3]:
# --- Notebook-wide config ---
PERSIST_DIR = "./chroma_srd"
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # fast/solid baseline
RERANK_MODEL_NAME = "BAAI/bge-reranker-large"  # or "cross-encoder/ms-marco-MiniLM-L-6-v2"

os.environ["OPENAI_API_KEY"] = "sk-..."

USE_OPENAI = True  # set True if you want OpenAI for generation

### Load SRD Data (Spells & Monsters)

In [5]:
# Kaggle raw file URLs (community mirrors; sometimes need Kaggle API if not public raw links)
SPELLS_CSV_URL = "https://raw.githubusercontent.com/wjsutton/games_night_viz/main/challenges/9_bonus_challenges/202206_datafamcon_dnd/dnd-spells.csv"
MONSTERS_CSV_URL = "https://raw.githubusercontent.com/wjsutton/games_night_viz/main/challenges/9_bonus_challenges/202206_datafamcon_dnd/dnd_monsters.csv"

spells_df = pd.read_csv(SPELLS_CSV_URL)
monsters_df = pd.read_csv(MONSTERS_CSV_URL)

print("Spells:", spells_df.shape)
print("Monsters:", monsters_df.shape)

Spells: (554, 12)
Monsters: (762, 17)


In [6]:
spells_df.head()

,name,classes,level,school,cast_time,range,duration,verbal,somatic,material,material_cost,description
0,Acid Splash,"Artificer, Sorcerer, Wizard",0,Conjuration,1 Action,60 Feet,Instantaneous,1,1,0,NaN,You hurl a bubble of acid. Choose one creature...
1,Blade Ward,"Bard, Sorcerer, Warlock, Wizard",0,Abjuration,1 Action,Self,1 round,1,1,0,NaN,You extend your hand and trace a sigil of ward...
2,Booming Blade,"Artificer, Sorcerer, Warlock, Wizard",0,Evocation,1 Action,Self (5-foot radius),1 round,0,1,1,a melee weapon worth at least 1 sp,You brandish the weapon used in the spell’s ca...
3,Chill Touch,"Sorcerer, Warlock, Wizard",0,Necromancy,1 Action,120 Feet,1 round,1,1,0,NaN,"You create a ghostly, skeletal hand in the spa..."
4,Control Flames,"Druid, Sorcerer, Wizard",0,Transmutation,1 Action,60 Feet,Instantaneous or 1 hour,0,1,0,NaN,You choose nonmagical flame that you can see w...


In [7]:
monsters_df.head()

,name,url,cr,type,size,ac,hp,speed,align,legendary,source,str,dex,con,int,wis,cha
0,aarakocra,https://www.aidedd.org/dnd/monstres.php?vo=aar...,1/4,humanoid (aarakocra),Medium,12,13,fly,neutral good,NaN,Monster Manual (BR),10.0,14.0,10.0,11.0,12.0,11.0
1,abjurer,NaN,9,humanoid (any race),Medium,12,84,NaN,any alignment,NaN,Volo's Guide to Monsters,NaN,NaN,NaN,NaN,NaN,NaN
2,aboleth,https://www.aidedd.org/dnd/monstres.php?vo=abo...,10,aberration,Large,17,135,swim,lawful evil,Legendary,Monster Manual (SRD),21.0,9.0,15.0,18.0,15.0,18.0
3,abominable-yeti,NaN,9,monstrosity,Huge,15,137,NaN,chaotic evil,NaN,Monster Manual,NaN,NaN,NaN,NaN,NaN,NaN
4,acererak,NaN,23,undead,Medium,21,285,NaN,neutral evil,NaN,Adventures (Tomb of Annihilation),NaN,NaN,NaN,NaN,NaN,NaN
